In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
import random
from pathlib import Path
from time import time
from datetime import datetime
from tqdm import tqdm, tnrange, tqdm_notebook
from pprint import pprint
from warnings import warn
from datetime import datetime
import itertools
from collections import Counter, OrderedDict
from bidi import algorithm as bidi
from hebrew_stopwords import hebrew_stopwords

In [5]:
mpl.rcParams.update({'font.size': 13})

In [6]:
DO_PARSE = False
SAVE_PARSER = False

# Load data

In [72]:
%%time
DATA = Path('Data')
meta = pd.read_csv(DATA/'meta.csv')
df = pd.read_csv(DATA/'data.csv')

CPU times: user 49.5 s, sys: 2.89 s, total: 52.4 s
Wall time: 50 s


In [8]:
if len(meta)!=len(np.unique(df.ID)):
    warn(f'Inconsistent dimensions! {len(meta):d}!={len(np.unique(df.ID)):d}')

# Knesset members

In [73]:
df.head()

,Unnamed: 0,header,body,ID
0,0,NaN,p_tamars\r\n\r\n2006-01-11Liraz\r\n\r\n\r\n\r\...,71333.0
1,1,סדר היום,א. קביעת מסגרת דיון להצעות להביע אי-אמון בממשל...,71333.0
2,2,"היו""ר רוני בר-און",בוקר טוב. אני מתכבד לפתוח את ישיבה מס' 74 של ו...,71333.0
3,3,זהבה גלאון,"אני מציעה ללכת על ההסדר הקבוע לדיון משולב, כמו...",71333.0
4,4,"היו""ר רוני בר-און","מה שנהגנו לעשות עד כה - המציעים, אחר-כך 20 דקו...",71333.0


In [74]:
meta.head()

,Unnamed: 0,Unnamed: 0.1,ID,CommitteeSessionID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,...,filesize,crc32c,error,i,date,committee_name,CommitteeID,CategoryDesc,CommitteeTypeDesc,AdditionalTypeDesc
0,0,0,71333,65782,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il//16/Committees/16_ptv...,2010-06-02 20:32:33,...,82050,LLmkqg==,NaN,0,2006-01-11,הכנסת,20,ועדת הכנסת,ועדת הכנסת,קבועה
1,1,1,71335,66045,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il//16/Committees/16_ptv...,2010-06-02 20:32:33,...,99662,wwtRjQ==,NaN,1,2006-01-11,הכנסת,20,ועדת הכנסת,ועדת הכנסת,קבועה
2,2,2,71336,66046,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il//16/Committees/16_ptv...,2010-06-02 20:32:34,...,7113,ti5ZGA==,NaN,2,2006-01-11,הכנסת,20,ועדת הכנסת,ועדת הכנסת,קבועה
3,3,3,71337,66050,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il//16/Committees/16_ptv...,2010-06-02 20:32:34,...,35967,Wyoukw==,NaN,3,2006-01-11,הכנסת,20,ועדת הכנסת,ועדת הכנסת,קבועה
4,4,4,71340,66166,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il//16/Committees/16_ptv...,2010-06-02 20:32:34,...,57224,K+cm8w==,NaN,4,2006-01-11,הכנסת,20,ועדת הכנסת,ועדת הכנסת,קבועה


In [75]:
df_counted = df.iloc[:,[1,2,3]].copy()

In [80]:
df_groupedby = df.groupby('header').agg({"body": lambda x: np.sum(x.str.len())})

In [81]:
df_groupedby

,body
header,
"! ^ ; ; . ; ??."" י- : -."". -"". ;",52.0
! אי אור,22.0
! י. פרץ,814.0
!-א' גולדשמ י דט,120.0
!ליד צאדק חאגי-יחיא,54.0
""" ""מוסד פיננסי שאינו משתתף"" – כל אחד מאלה:",494.0
""" ""מנוי"" – אחד מאלה, לפי העניין",2308.0
""" ביילין",21.0
""" דורון לוי",87.0
